# Compiling and running Cython code
Like C and C++, Cython requires a compilation step before source can be run. This compilation step can be explicit or implicit. There are several compilation options...

## The standard way: using distutils with cythonize
Of particular interest to us is the ability of distutils to compile C source into an extension module.

In [1]:
file_name = "fib.pyx"

content = """
def fib(long n):
    '''Returns the nth Fibonacci number.'''
    cdef long a=0, b=1, i
    for i in range(n):
        a, b = a + b, a
    return a
"""

with open(file_name, "w") as target_file:
    target_file.write(content)


In [2]:
%%bash
cat fib.pyx


def fib(long n):
    '''Returns the nth Fibonacci number.'''
    cdef long a=0, b=1, i
    for i in range(n):
        a, b = a + b, a
    return a


You may be plagued by the issue ['error: Unable to find vcvarsall.bat' when compiling Cython code](https://stackoverflow.com/questions/53172601/error-unable-to-find-vcvarsall-bat-when-compiling-cython-code). Installing the [Microsoft Visual C++ Compiler for Python 2.7](https://www.microsoft.com/en-us/download/confirmation.aspx?id=44266) and including 'import setuptools' at the top of setup.py appears to have resolved the issue.


In [3]:
file_name = "setup.py"

content = """
import setuptools  # important
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules=cythonize('fib.pyx'))
"""

with open(file_name, "w") as target_file:
    target_file.write(content)


In [1]:
%%bash
activate py36
python setup.py build

running build
running build_ext


There will now be a fib.pyd file that we can use...

In [4]:
import os

file_path = os.path.join(".", "build", "lib.win-amd64-3.6", "fib.cp36-win_amd64.pyd")
os.path.isfile(file_path)

True

Capy this file into the local directory as fib.pyd and import it...

In [1]:
import fib

In [2]:
fib.fib

<function fib.fib>

In [3]:
fib.fib(10)

55

In [14]:
x = int(1e6)

In [29]:
%%timeit
[fib.fib(i) for i in range(1, 11)]

1.14 µs ± 45.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
def pyfib(n):
    a=0; b=1;
    for i in range(n):
        a, b = a + b, a
    return a


In [28]:
%%timeit
[pyfib(i) for i in range(1, 11)]

5.85 µs ± 396 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


So the Cython version is around 5 times faster in this case.

## %%cython magic

In [1]:
%load_ext Cython

In [2]:
%%cython
def fib(int n):
    cdef int i
    cdef double a=0.0, b=1.0
    for i in range(n):
        a, b = a+b,a
    return a


In [4]:
fib(90)

2.880067194370816e+18

## Compiling on the fly with pyximport

In [5]:
import pyximport

In [6]:
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x1cadcf8b550>)

In [7]:
code = """
def fib(int n):
    cdef int i
    cdef double a=0.0, b=1.0
    for i in range(n):
        a, b = a+b,a
    return a
"""

file_name = "cyfib.pyx"
with open(file_name, "w") as target_file:
    target_file.write(code)


In [8]:
import cyfib

In [10]:
cyfib.fib

<function cyfib.fib>

In [11]:
fib(90)

2.880067194370816e+18

Because Cython modules imported via pyximport depend on both the cython compiler and a properly set up C compiler, it tends not to be used in production environments where these dependencies are not under our control.

***